In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os, sys
# py_file_location = '/content/drive/MyDrive/LMU New/SoSe2022/Unsupervised DL/self_supervised'
# sys.path.append(os.path.abspath(py_file_location))
# %pwd

# %cd '/content/drive/MyDrive/LMU New/SoSe2022/Unsupervised DL/self_supervised'

In [7]:
!pip install pytorch_lightning
!pip install torchmetrics

In [5]:
%cd '/Users/seungheejeong/Desktop/self_supervised'
!pip install -r requirements.txt

/Users/seungheejeong/Desktop/self_supervised


In [10]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
import pytorch_lightning as pl

import pandas as pd
import matplotlib.pyplot as plt 

# make the plots pretty
import matplotlib as mpl
from matplotlib import rc

from model import SelfSupervisedMethod
from model_params import VICRegParams

OSError: dlopen(/Users/seungheejeong/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/torchtext/lib/libtorchtext.so, 0x0006): Symbol not found: __ZN3c1012OptionalType3getENS_4Type24SingletonOrSharedTypePtrIS1_EE
  Referenced from: /Users/seungheejeong/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/torchtext/lib/libtorchtext.so
  Expected in: /Users/seungheejeong/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib

In [ ]:
# Dataset and dataloader
# train_img_path = py_file_location+'/data/C. Diabetic Retinopathy Grading/train_data/train/Training_Set'
# val_img_path = py_file_location+'/data/C. Diabetic Retinopathy Grading/train_data/val/Training_Set'
# csv_file_path = py_file_location+'/data/C. Diabetic Retinopathy Grading/2. Groundtruths/a. DRAC2022_ Diabetic Retinopathy Grading_Training Labels.csv'

DEVICE = 'cuda'

metric_collection = MetricCollection([
    Accuracy(),
    Precision(num_classes=15, average='macro'),
    Recall(num_classes=15, average='macro')
])

In [ ]:
# =============================================================================
#use sgd with smaller lr
# don't use pretrained model and see what is the output of the model
# epoch 200 ~ 300


hparams = VICRegParams(
  # encoder_arch='resnet50',
   dataset_name="challenge",
   transform_apply_blur=False,
   mlp_hidden_dim=2048,
   dim=2048,
   batch_size=256, # 왜 512*2048이랑 256*2048 문제가 생기는가
   lr=0.001,
   optimizer_name= "sgd",
   final_lr_schedule_value=0,
   weight_decay=1e-4,
)

logger = CSVLogger(py_file_location, name="logger")

# epoch 200~300
model = SelfSupervisedMethod(hparams)
trainer = pl.Trainer(
    gpus=1,
    accelerator="auto",
    logger=logger,
    max_epochs=10,
    callbacks=[EarlyStopping(monitor="step_train_loss", stopping_threshold=26)]
    )
trainer.fit(model)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 

# make the plots pretty
import matplotlib as mpl
from matplotlib import rc
mpl.rcParams['mathtext.default']='regular'
plt.rc('font', family='serif', size=16)
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12) 

In [ ]:
run = pd.read_csv('/content/drive/MyDrive/LMU New/SoSe2022/Unsupervised DL/self_supervised/logger.csv')

In [ ]:
plt.plot(run['step'], run['loss'],  color='red')
plt.ylabel('Loss')
plt.xlabel('Step')
plt.title('VICReg')
plt.xlim([0,4000])